In [ ]:
import sys
sys.path.append('../../evapo_scripts/Auxiliary/')
from helpsters import *

In [ ]:
# workaround with relative path mess from containers
os.chdir('/data/Aldhani/eoagritwin/')
# get rid of force output that is not needed -> months outside of growing season that do not exist in AI4Boundaries
files = getFilelist('/data/Aldhani/eoagritwin/force/output/BRANDENBURG/', '.tif', deep=True)

filesToKill = [f for f in files if int(f.split('-')[-1].split('.')[0]) not in [i for i in range(3,9,1)]]
for file in filesToKill:
    RasterKiller(file)

files = list(filter(lambda item: item not in filesToKill, files))

In [ ]:
# bring files in list in order
tiles = list(set([file.split('output/')[-1].split('/')[1].split('/')[0] for file in files]))

# subset for test!!!!!!!!!!!!!!!!
tiles = ['X0068_Y0042','X0069_Y0042']
####################################


tilefilesL = []
for tile in tiles:
    tilefiles = [file for file in files if tile in file]
    tilefilesL.append(getBluGrnRedBnrFORCEList(tilefiles))

# create stacked vrts of tiles
# make folder for tiles
outDir = f'/data/Aldhani/eoagritwin/fields/Auxiliary/vrt/{getFORCExyRange(tiles)}/'
os.makedirs(outDir, exist_ok=True)

for i in range(len(tilefilesL[0])):
    vrt = gdal.BuildVRT(f'{outDir}{getFORCExyRange(tiles)}_{str(i)}.vrt', [tilefile[i] for tilefile in tilefilesL], separate = False)
    vrt = None

# make paths in vrts relative
vrts = getFilelist(outDir, '.vrt')
for vrt in vrts:
    convertVRTpathsTOrelative(vrt)
nums = [int(vrt.split('_')[-1].split('.')[0]) for vrt in vrts]
vrts_sorted = sortListwithOtherlist(nums, vrts)[-1]

vrt = gdal.BuildVRT(f'{outDir}{getFORCExyRange(tiles)}_Cube.vrt', vrts_sorted, separate = True)
vrt = None
convertVRTpathsTOrelative(f'{outDir}{getFORCExyRange(tiles)}_Cube.vrt')

In [ ]:
# make vrt of predicted image chips
size = str(128*2)
over = str(20)
chips = getFilelist('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/chips/', '.tif')
chips = [chip for chip in chips if size in chip and over in chip]

for c in chips:print(c)
# create stacked vrts of chips
vrt = gdal.BuildVRT(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/{size}_{over}_chipsvrt.vrt', chips, separate = False)
vrt = None
convertVRTpathsTOrelative(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/{size}_{over}_chipsvrt.vrt')


In [ ]:
# make vrt of masked predicted image chips
chips = getFilelist('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/masked_chips/', '.tif')
chips = [chip for chip in chips if size in chip and over in chip]
# create stacked vrts of masked chips
vrt = gdal.BuildVRT(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/{size}_{over}_masked_chipsvrt.vrt', chips, separate = False)
vrt = None
convertVRTpathsTOrelative(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/{size}_{over}_masked_chipsvrt.vrt')


In [41]:
mean_s2 = np.array([5.4418573e+02, 7.6761194e+02, 7.1712860e+02, 2.8561428e+03 ]).astype(np.float32) 
std_s2  = np.array( [3.7141626e+02, 3.8981952e+02, 4.7989127e+02 ,9.5173022e+02]).astype(np.float32) 

d1 = np.asarray([i for i in range(4*6*10*10)]).astype(np.float32)
temp = d1.reshape(4,6,10,10)
print(temp.shape)
print(temp[2,0,8,0])
temp2 = temp.T

temp2 -= mean_s2

temp2 /= std_s2

temp = temp2.T

print(temp[2,0,8,0])


(4, 6, 10, 10)
1280.0
1.1729145


In [42]:
(1280-mean_s2[2])/std_s2[2]

np.float32(1.1729145)